# <center> Post Modern Portfolio Theory </center>
### <center> Zhangyi Hu </center>
<br>


# <center> Contents </center>
- Overview on Markowitz mean variance model (MPT)
- Stochastic Dominance and Post Modern Portfolio Theory
- Stochastic programming optimization approach
- Python package PyMCEF

# <center> Modern Portfolio Theory </center>


- Introduced by Harry Markowitz in 1952, lead to Nobel price in economics
- Portfolio selection is based the first two moments of the random return
- Also known as mean variance model

### Introduction

Suppose there are $N$ assets available for investment. 
A portfolio associated with that is a vector with $N$ elements, summing up to 1. 
The set of all possible portfolio is defined as:

$$\mathcal{W}=\left\{ \boldsymbol{w}:\,\boldsymbol{w}\in R^{N},\sum_{n=1}^{N}w_{n}=1\right\}$$ 

Let's denote the realization of the percentage return of all $N$ assets as a vector $\boldsymbol{Y}$.
Then the Reward of this portfolio is defined as its expected return.

$$E[X(\boldsymbol{w})]=E[\boldsymbol{Y}\cdot\boldsymbol{w}]=E[\sum_{n=1}^{N}w_{n}Y_{n}]$$

A portfolio is called efficient, if, with its reward fixed, the risk is minimized, or with its risk fixed, the reward is maximized. 
The set of all possible efficient portfolios for given $\boldsymbol{Y}$ is defined as the efficient frontier. 
$\boldsymbol{Y}$ can be formulated from a distribution, (e.g. by providing the means and covariance matrix). The distribution can be provided by various methods, with certain assumptions made.